## 准备数据

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [5]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([28*28+1, 100]))  
        self.W2 = tf.Variable(tf.random.normal([100, 10]))
        self.b1 = tf.Variable(np.zeros(100), dtype=tf.float32)  # 使用 tf.Variable 来定义偏置项 b1
        self.b2 = tf.Variable(np.zeros(10), dtype=tf.float32)  
        
        
        
        
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = tf.reshape(x, [-1, 28*28])  # 展平每个图像
        bias = tf.ones(shape=[x.shape[0], 1], dtype=tf.float32)  # 生成偏置项
        x = tf.concat([x, bias], axis=1)  # 将偏置项拼接到输入
        
        self.h1 = tf.matmul(x, self.W1)+self.b1 # shape(5, 4)
        self.h1_relu = tf.nn.relu(self.h1)
        self.h2 = tf.matmul(self.h1_relu, self.W2)+self.b2
        return self.h2
        
model = myModel()
optimizer = optimizers.Adam()

## 计算 loss

In [7]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [8]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 145.14337 ; accuracy 0.101616666
epoch 1 : loss 125.35512 ; accuracy 0.10896666
epoch 2 : loss 112.016014 ; accuracy 0.11625
epoch 3 : loss 102.329956 ; accuracy 0.12165
epoch 4 : loss 94.76663 ; accuracy 0.123966664
epoch 5 : loss 88.51399 ; accuracy 0.12625
epoch 6 : loss 83.15317 ; accuracy 0.12708333
epoch 7 : loss 78.46095 ; accuracy 0.12878333
epoch 8 : loss 74.302155 ; accuracy 0.12993333
epoch 9 : loss 70.61248 ; accuracy 0.13125
epoch 10 : loss 67.34032 ; accuracy 0.13181667
epoch 11 : loss 64.45048 ; accuracy 0.134
epoch 12 : loss 61.903965 ; accuracy 0.1359
epoch 13 : loss 59.6561 ; accuracy 0.13851666
epoch 14 : loss 57.66418 ; accuracy 0.14173333
epoch 15 : loss 55.88426 ; accuracy 0.14601667
epoch 16 : loss 54.27765 ; accuracy 0.15025
epoch 17 : loss 52.803493 ; accuracy 0.15473333
epoch 18 : loss 51.438274 ; accuracy 0.15901667
epoch 19 : loss 50.16747 ; accuracy 0.16341667
epoch 20 : loss 48.97705 ; accuracy 0.16863333
epoch 21 : loss 47.860092 ; accuracy